In [1]:
import os
import parameter
import pandas as pd
import simplejson
import pickle
from cluster_model import ClusterModel
from datetime import timedelta

In [2]:
with open('../data/station_data/2024 model/v2/tph_4/data.pickle', 'rb') as f:
    station_data = pickle.load(f)

with open('../station_information.json', 'r') as f:
    station_information = simplejson.load(f)['data']['stations']
    
model = ClusterModel(station_data=station_data, in_transit=[], tph=4)
model_2 = ClusterModel(station_data=station_data, in_transit=[], tph=4)
model.init_clusters(0.005)

36 horizontal squares and 51 vertical squares. Total squares: 1836


In [ ]:
day = 8
files = os.listdir(f'rebalancing/data/{day}')
files.sort()
for i, file in enumerate(files):
    df = parameter.get_state_df(station_information, f'rebalancing/data/{day}/' + file,)
    split_file = file[:-7].split('-')
    hour = int(split_file[3])-4
    minute = int(split_file[4])
    model.init_df_state(df, time=timedelta(hours=(hour+minute/60)))
    


In [ ]:
def detect_rebalance(model):
    for cluster in model.cluster_dict.values():
        